In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np
import datetime
import sqlite3

/var/folders/5l/zg92rvh50792032mpqnwpydw0000gn/T/ipykernel_28556/3836467874.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


1) Define a databank table with the available drugs to be prescribed. For the sake of simplicity take the unique ones from the same source, that will be also used for the interaction detection.

In [2]:
sparql = SPARQLWrapper("https://bio2rdf.org/sparql")

query = """
PREFIX dv: <http://bio2rdf.org/drugbank_vocabulary:>
PREFIX brv: <http://bio2rdf.org/bio2rdf_vocabulary:>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT DISTINCT ?id ?name
WHERE {
  ?escitalopram a dv:Drug .
  ?escitalopram brv:identifier ?id .
  ?escitalopram dct:title ?name
  FILTER(LANG(?name) = "en") .
}
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
ddi_df = sparql.query().convert()

data = []

for r in ddi_df["results"]["bindings"]:
    drug_id = r["id"]["value"]
    drug_name = r["name"]["value"]
    data.append((drug_id, drug_name))

drug_df = pd.DataFrame(data, columns=['id', 'drug_name'])
drug_df.head()

,id,drug_name
0,DB00088,Alglucerase
1,DB00089,Capromab
2,DB00090,Laronidase
3,DB00091,Cyclosporine
4,DB00092,Alefacept


In [3]:
db = sqlite3.connect('ddi.db')
cursor = db.cursor()
cursor.execute('''
    DROP TABLE IF EXISTS drugs;
''')
cursor.execute('''
    CREATE TABLE drugs (
        id TEXT PRIMARY KEY,
        drug_name TEXT NOT NULL
    );
''')
drug_df.to_sql('drugs', db, if_exists='replace', index=False)
db.commit()
cursor.execute('SELECT * FROM drugs LIMIT 10')
print(cursor.fetchall())

[('DB00088', 'Alglucerase'), ('DB00089', 'Capromab'), ('DB00090', 'Laronidase'), ('DB00091', 'Cyclosporine'), ('DB00092', 'Alefacept'), ('DB00093', 'Felypressin'), ('DB00094', 'Urofollitropin'), ('DB00095', 'Efalizumab'), ('DB00096', 'Serum albumin'), ('DB00097', 'Choriogonadotropin alfa')]


2) Define patient table as an oversimplified version of any electronic patient management system

In [4]:
db = sqlite3.connect('ddi.db')
cursor = db.cursor()
cursor.execute('''
    DROP TABLE IF EXISTS patients;
''')
cursor.execute('''
     CREATE TABLE patients (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        birth_date DATE NOT NULL,
        name TEXT NOT NULL
    );
''')
cursor.execute('INSERT INTO patients (birth_date, name) VALUES (?, ?)', (datetime.datetime(1997, 7, 18), "Max Mustermann"))
cursor.execute('INSERT INTO patients (birth_date, name) VALUES (?, ?)', (datetime.datetime(1994, 2, 24), "Erika Musterfrau"))
db.commit()
cursor.execute('SELECT * FROM patients')
print(cursor.fetchall())
db.close()

[(1, '1997-07-18 00:00:00', 'Max Mustermann'), (2, '1994-02-24 00:00:00', 'Erika Musterfrau')]


3) Define prescriptions table as a simulation of any e-prescription extension of the above system

In [5]:
db = sqlite3.connect('ddi.db')
cursor = db.cursor()
cursor.execute('''
    DROP TABLE IF EXISTS prescriptions;
''')
cursor.execute('''
    CREATE TABLE prescriptions (
        patient_id INTEGER NOT NULL,
        drug_id TEXT NOT NULL,
        FOREIGN KEY (patient_id) REFERENCES patients(id),
        FOREIGN KEY (drug_id) REFERENCES drugs(id)
    );
''')
cursor.execute('INSERT INTO prescriptions (patient_id, drug_id) VALUES (?, ?)', (1, "DB01175"))
cursor.execute('INSERT INTO prescriptions (patient_id, drug_id) VALUES (?, ?)', (2, "DB01012"))
db.commit()

cursor.execute('SELECT * FROM prescriptions')
print(cursor.fetchall())

db.close()

[(1, 'DB01175'), (2, 'DB01012')]


Query 1: Sample query for interaction within the tool

In [6]:
def get_db_connection():
    conn = sqlite3.connect('ddi.db')
    return conn

conn = get_db_connection()
patients = conn.execute('''
               SELECT p.drug_id, d.drug_name
               FROM prescriptions p
               INNER JOIN drugs d ON p.drug_id = d.id
               WHERE p.patient_id = 1
''').fetchall()
conn.close()

Query 2: DDI Query to used as a service in the tool

In [7]:
sparql = SPARQLWrapper("https://bio2rdf.org/sparql")

drug_id = "DB01175"

query = f"""
PREFIX dv: <http://bio2rdf.org/drugbank_vocabulary:>
PREFIX brv: <http://bio2rdf.org/bio2rdf_vocabulary:>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT DISTINCT ?object
WHERE {{
  ?escitalopram a dv:Drug .
  ?escitalopram brv:identifier ?id .
  ?escitalopram dv:ddi-interactor-in ?drug_interaction .
  ?drug_interaction brv:identifier ?object .
  FILTER(STR(?id) = "{drug_id}") .
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
ddi_df = sparql.query().convert()

data = []

for r in ddi_df["results"]["bindings"]:
    drug_ids = r["object"]["value"]
    data.append(drug_ids)

ddi_res = pd.DataFrame(data, columns=['ddi'])
ddi_res = ddi_res['ddi'].str.split('_').explode().unique()